In [162]:
import os
import sys

module_path = os.path.abspath(os.path.join('../..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

/home/enis/projects/nna/src


In [163]:
from params import PRE_PROCESSED_queue,VGGISH_processing_queue,VGGISH_EMBEDDINGS_queue
from params import PRE_PROCESSED_queue,PRE_PROCESSING_queue,VGGISH_processing_queue
from pathlib import Path
import pre_process_func
from pre_process_func import read_queue

import pprint
pp = pprint.PrettyPrinter(indent=4)
pprint = pp.pprint

PRE_PROCESSED_queue = "../../" / Path(PRE_PROCESSED_queue)
VGGISH_processing_queue = "../../" / Path(VGGISH_processing_queue)
PRE_PROCESSING_queue = "../../" / Path(PRE_PROCESSING_queue)
VGGISH_EMBEDDINGS_queue = "../.." / Path(VGGISH_EMBEDDINGS_queue)

In [164]:
PRE_PROCESSED_queue,VGGISH_processing_queue

(PosixPath('../../job_logs/pre_processed_queue.csv'),
 PosixPath('../../job_logs/VGGISH_processing_queue.csv'))

In [165]:
VGGISH_EMBEDDINGS_queue

PosixPath('../../job_logs/vggish_embeddings_queue.csv')

In [184]:
already_PRE_PROCESSING=read_queue(PRE_PROCESSING_queue)
files_in_pre_queu=read_queue(PRE_PROCESSED_queue)
files_in_processing=read_queue(VGGISH_processing_queue)
files_done=read_queue(VGGISH_EMBEDDINGS_queue)
files_to_do = set(files_in_pre_queu).difference(set(files_in_processing),set(files_done))
files_doing=set(files_in_processing).difference(files_done)

print("started pre processing",len(set(already_PRE_PROCESSING)))
print("pre processing done",len(set(files_in_pre_queu)))
print("VGG","doing:",len(files_doing), "to do",len(files_to_do))
print("files done",len(set(files_done)),"{:.4}%".format(len(set(files_done))/16739.0))

# pprint(files_in_processing)

started pre processing 63
pre processing done 59
VGG doing: 0 to do 59
files done 16676 0.9962%


In [14]:
apath=Path('/home/data/nna/stinchcomb/NUI_DATA/18 Fish Creek 4/July 2016/FSHCK4_20160629_194935.MP3')
str(apath) in already_PRE_PROCESSING

True

In [49]:
len(set(already_PRE_PROCESSING)),len((already_PRE_PROCESSING))

(367, 617)

#### remove wrong file names

In [81]:
import csv
from pre_process_func import save_to_csv

with open("job_logs/pre_processed_queue_old.csv") as f:
    reader=csv.reader(f)
    lines=[]
    for i in reader:
        if len(i[0])<10:
            print(i)
        else:
            lines.append(i)
save_to_csv("job_logs/pre_processed_queue.csv",lines)

['npy']


#### check if all results have continous indexes 

In [99]:
from pre_process_func import OUTPUT_DIR


In [137]:
all_files=Path(OUTPUT_DIR).iterdir()

<generator object Path.iterdir at 0x7fc07b4107c8>

In [243]:
# THIS checks if all segment files are exists in order (splits of original audio files),
# if last file missing, cannot tell
def check_order(folder):
    all_files=list(Path(folder).iterdir())
#     print(list(all_files))
    all_folders = [i for i in all_files if i.is_dir()]
    all_non_folders = [i for i in all_files if not(i.is_dir())]
#     print(all_non_folders)
    i_raw=[]
    i_embed=[]
    total=0
    for afile in sorted(all_non_folders):
        index=afile.stem.split("_")[-1]
        if "rawembeddings" in index:
            rawembed_index= int(index[-3:])
            i_raw.append(rawembed_index)
        elif "embeddings" in index:
            embed_index= int(index[-3:])
            i_embed.append(embed_index)
        elif ".txt" in str(afile) or "preprocessed.npy" in str(afile):
            continue
        else:
            print(index)
            print(afile)
#     if len(i_raw)<49 and i_raw:
#         print(i_raw,i_embed)
    total+=len(i_raw)
    if i_raw:
        if len(i_raw)!=i_raw[-1]+1:
            for i in range(len(i_raw)):
                if i!=i_raw[i]:
                    print(i)
                    break
            print(i_raw)
            print(afile)
    if i_embed:
        if len(i_embed)!=i_embed[-1]+1:
            for i in range(len(i_embed)):
                if i!=i_embed[i]:
                    print(i)
                    break
            print(i_embed)
            print(afile)
    
    for folder in all_folders:
        total+=check_order(folder)
    return total

In [244]:

print(check_order(OUTPUT_DIR))

40
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49]
/scratch/enis/data/nna/NUI_DATA/17 Fish Creek 3/July 2016/FSHCK3_20160625_153654_vgg/FSHCK3_20160625_153654_rawembeddings049.npy
40
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49]
/scratch/enis/data/nna/NUI_DATA/17 Fish Creek 3/July 2016/FSHCK3_20160625_153654_vgg/FSHCK3_20160625_153654_rawembeddings049.npy
18907


In [246]:
# check if missing file was in the pre_queue
apath="/scratch/enis/data/nna/NUI_DATA/17 Fish Creek 3/July 2016/FSHCK3_20160625_153654_vgg/FSHCK3_20160625_153654_rawembeddings047.npy" 
# /scratch/enis/data/nna/NUI_DATA/04 Colville 4/June 2016/CLVL4_20160623_084159_preprocessed/output018_preprocessed.npy
str(apath) in files_in_pre_queu

False

In [247]:
# expected format in that list
files_in_pre_queu[0]

'/scratch/enis/data/nna/NUI_DATA/02 Colville 2/August 2016/CLVL2_20160827_085325_preprocessed/output000_preprocessed.npy'

'/scratch/enis/data/nna/NUI_DATA/02 Colville 2/August 2016/CLVL2_20160827_085325_preprocessed/output000_preprocessed.npy'

In [296]:
# check if files exists, maybe just not recorded
!ls -alh "/scratch/enis/data/nna/NUI_DATA/17 Fish Creek 3/July 2016/FSHCK3_20160625_153654_preprocessed/output040_preprocessed.npy"


-rw-rw-r-- 1 enis enis 85M Oct  9 05:20 /scratch/enis/data/nna/NUI_DATA/17 Fish Creek 3/July 2016/FSHCK3_20160625_153654_preprocessed/output040_preprocessed.npy


In [294]:
!ls -alh "/scratch/enis/data/nna/NUI_DATA/17 Fish Creek 3/July 2016/FSHCK3_20160625_153654_preprocessed/output040_preprocessed.npy"

-rw-rw-r-- 1 enis enis 85M Oct  9 05:20 /scratch/enis/data/nna/NUI_DATA/17 Fish Creek 3/July 2016/FSHCK3_20160625_153654_preprocessed/output040_preprocessed.npy


In [298]:
# check which stages file was in 
str(a_path) in files_in_pre_queu, str(a_path) in files_in_processing, str(a_path) in files_done, 

(False, False, False)

### find out which files are not processed 

In [52]:
# files_done format
files_done[0]

'/scratch/enis/data/nna/real/39B/S4A10262_20190604_134602_preprocessed/output000_preprocessed.npy'

In [53]:
# format
all_raw[0]

'/scratch/enis/data/nna/real/11A/S4A10276_20190507_000000_vgg/S4A10276_20190507_000000_rawembeddings000.npy'

set()

In [186]:
# get file names that were supposed to be processed
with open("/home/enis/projects/nna/data/flacfiles.txt","r+") as my_file:
    input_path_list = my_file.read().splitlines()

In [187]:
# get file stems, ignore directories, assumes that all file names are unique 
files_donestem=(set(["_".join(str(Path(i).parent.stem).split("_")[:-1]) for i in files_done]))
files_in_pre_queu_stem=(set(["_".join(str(Path(i).parent.stem).split("_")[:-1]) for i in files_in_pre_queu]))


all_rawstem=(set(["_".join(Path(i).stem.split("_")) for i in input_path_list]))
pre_procesing_stem=(set(["_".join(Path(i).stem.split("_")) for i in already_PRE_PROCESSING]))



In [188]:
leftones=all_rawstem.difference(files_donestem.union(set(files_in_pre_queu_stem)))

In [189]:
leftones_path=[]
for i in input_path_list:
    short="_".join(Path(i).stem.split("_"))
    if short in leftones:
        leftones_path.append(i)

In [190]:
leftones_path

['/tank/data/nna/real/28A/S4A10259_20190514_183000.flac',
 '/tank/data/nna/real/48A/S4A10252_20190529_171602.flac',
 '/tank/data/nna/real/15A/S4A10283_20190528_030000.flac',
 '/tank/data/nna/real/33A/S4A10287_20190522_014602.flac']

In [99]:
# save them to a file for further processing
with open("/home/enis/projects/nna/data/flacfiles_left.txt","w") as my_file:
    for path in leftones_path:
        my_file.write(path+"\n")